In [2]:
import pandas as pd
import numpy as np

In [115]:
pd.options.mode.chained_assignment = None  # default='warn'


In [116]:
zri_acs_bike = pd.read_csv('../data/zri_acs_bikeshare_merged.csv')

In [117]:
zri_acs_bike.shape

(93672, 51)

In [118]:
bds = pd.read_csv('../data/bds_engineered_features.csv',index_col=0)

In [119]:
bds.shape

(93744, 12)

In [120]:
zri_acs_bike.head(2)

,zip,City,State,Metro,CountyName,zri,year,month,datetime,percent_white,...,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
0,1013,Chicopee,MA,Springfield,Hampden County,928.0,2014,1,2014-01-01,0.728315,...,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
1,1013,Chicopee,MA,Springfield,Hampden County,931.0,2014,2,2014-02-01,0.727052,...,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0


In [121]:
bds.head(2)

,zip,City,State,Metro,CountyName,year-month,zri,year,total_firms,job_creation_rate,job_destruction_rate,startup_firms
0,1013,Chicopee,MA,Springfield,Hampden County,2014-01-01,928.0,2014,7376,9.972,9.108,393
1,1013,Chicopee,MA,Springfield,Hampden County,2014-02-01,931.0,2014,7376,9.972,9.108,393


In [123]:
new_bds = bds.copy()
for z in new_bds['zip'].unique():
    temp = new_bds[new_bds['zip']==z]
    rows_to_interpolate = temp['year-month'].map(lambda row: row not in ['2014-01-01','2015-01-01',
                                                      '2016-01-01','2017-01-01',
                                                      '2018-01-01','2019-01-01',
                                                      '2019-12-01'])
    temp.loc[rows_to_interpolate,
             ['total_firms','job_creation_rate','job_destruction_rate','startup_firms']] = np.nan
    temp = temp.interpolate(method='linear')
    new_bds.loc[new_bds['zip']==z] = temp
    

In [129]:
new_bds

,zip,City,State,Metro,CountyName,year-month,zri,year,total_firms,job_creation_rate,job_destruction_rate,startup_firms
0,1013,Chicopee,MA,Springfield,Hampden County,2014-01-01,928.0,2014,7376.0,9.97200,9.108000,393.000000
1,1013,Chicopee,MA,Springfield,Hampden County,2014-02-01,931.0,2014,7373.0,9.99275,9.034667,392.166667
2,1013,Chicopee,MA,Springfield,Hampden County,2014-03-01,934.0,2014,7370.0,10.01350,8.961333,391.333333
3,1013,Chicopee,MA,Springfield,Hampden County,2014-04-01,929.0,2014,7367.0,10.03425,8.888000,390.500000
4,1013,Chicopee,MA,Springfield,Hampden County,2014-05-01,929.0,2014,7364.0,10.05500,8.814667,389.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
93739,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-08-01,1277.0,2019,1750.0,16.77200,16.359000,161.000000
93740,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-09-01,1271.0,2019,1750.0,16.77200,16.359000,161.000000
93741,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-10-01,1299.0,2019,1750.0,16.77200,16.359000,161.000000
93742,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-11-01,1261.5,2019,1750.0,16.77200,16.359000,161.000000


In [130]:
zri_acs_bike_bds = pd.merge(zri_acs_bike,new_bds,how='left',left_on=['zip','State','CountyName','zri','datetime'],
         right_on=['zip','State','CountyName','zri','year-month'])

In [133]:
zri_acs_bike_bds = zri_acs_bike_bds.drop(['City_y','Metro_y','year-month','year_y'],axis=1)

In [136]:
zri_acs_bike_bds.columns = ['zip', 'City', 'State', 'Metro', 'CountyName', 'zri', 'year',
       'month', 'datetime', 'percent_white', 'percent_black', 'percent_asian',
       'percent_hispanic', 'percent_native_am', 'percent_other_race',
       'percent_0_17', 'percent_18_39', 'percent_40_64', 'percent_65+',
       'percent_rental_units_vacant', 'percent_rental_units_occupied',
       'percent_graduate_deg', 'percent_bachelors', 'percent_associates',
       'percent_highschool', 'percent_less_highschool',
       'percent_commute_public_transport', 'percent_commute_less_30',
       'percent_buildings_less_10_units', 'percent_buildings_10_19_units',
       'percent_buildings_20_49_units', 'percent_buildings_50+_units',
       'percent_commute_30_to_59', 'percent_commute_60_to_89',
       'percent_commute_90_more', 'percent_new_city', 'percent_new_unit',
       'percent_units_owner_occupied', 'median_building_age',
       'income_per_capita', 'poverty_rate', 'total_pop',
       'percent_workforce_unemployed', 'percent_work_from_home', 'median_age',
       'percent_female', 'gini_index', 'percent_not_us_citizen',
       'bs_total_stations', 'bs_total_systems', 'has_bike_sharing',
       'total_firms', 'job_creation_rate', 'job_destruction_rate',
       'startup_firms']

In [141]:
zri_acs_bike_bds.to_csv('./../data/zri_acs_bikeshare_bds_merged.csv')